This is a synthetic dataset that is based on a real dataset. These datasets are based on Kaggle's May 2021 tabular competition.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Read the files

In [ ]:
#Reading train file:
train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv')
#Reading test file:
test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv')
#reading sample submission
submission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv')


In [ ]:
train

In [ ]:
test

In [ ]:
submission

Check for null values

In [ ]:
train.isnull().sum().sum()

In [ ]:
test.isnull().sum().sum()

Analyse target

In [ ]:
import matplotlib.pyplot as plt

train.groupby('target').target.count().plot.bar(ylim=0)
plt.show()


Map target

In [ ]:
classes = {"Class_1": 1, "Class_2": 2, "Class_3": 3, "Class_4":4}

train.target = train.target.map(classes)
train.target

Drop target from train

In [ ]:
target = train.target

train.drop('target', axis = 1, inplace = True)
train

Combine train and test

In [ ]:
combi = train.append(test)
combi

Drop ID from combi

In [ ]:
combi.drop('id', axis = 1, inplace = True)
combi

Normalise combi

In [ ]:
combi = (combi.max() - combi) / (combi.max() - combi.min())
combi

Define X and y

In [ ]:
y = target
X = combi[: len(train)]
X_test = combi[len(train) :]

In [ ]:
y.shape

In [ ]:
X.shape

In [ ]:
X_test.shape

SMOTE

In [ ]:
pip install smote-variants

In [ ]:
import smote_variants as sv

oversampler= sv.MulticlassOversampling(sv.distance_SMOTE())
X_samp, y_samp= oversampler.sample(X, y)

In [ ]:
X_samp.shape, y_samp.shape

Split into training and validation

In [ ]:
#split train set for testing
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_samp, y_samp, test_size=0.10, random_state=1, stratify=y_samp)
X_train.shape, y_train.shape, X_val.shape, y_train.shape, X_test.shape

Select model

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=100, class_weight= 'balanced', multi_class='multinomial', random_state=1, max_iter=20000).fit(X_train, y_train)
model.score(X_train, y_train)

Predict on validation set

In [ ]:
y_pred = model.predict(X_val)
model.score(X_val, y_val)

In [ ]:
y_pred

Predict on test set

In [ ]:
prediction = model.predict(X_test)
prediction.shape

One hot encode prediction

In [ ]:
predictions = pd.get_dummies(prediction)
predictions.columns=['Class_1', 'Class_2', 'Class_3', 'Class_4']
predictions

Prepare submission

In [ ]:
submission.Class_1 = predictions.Class_1
submission.Class_2 = predictions.Class_2
submission.Class_3 = predictions.Class_3
submission.Class_4 = predictions.Class_4

In [ ]:
submission.to_csv('submission.csv', index=False)
submission = pd.read_csv("submission.csv")
submission
